# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [20]:
#Your code here

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
%matplotlib inline

df = pd.read_csv('homepage_actions.csv')
print(len(df))
df.head()





8188


,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [21]:
#Your code here
c = set(df[df.action=='click']['id'].unique())
v = set(df[df.action=='view']['id'].unique())
print("views:",len(v),"clicks:", len(c))
print("views who didn't click:",len(v-c))
print("clicks who didn't view:",len(c-v))


e = set(df[df.group=='experiment']['id'].unique())
c = set(df[df.group=='control']['id'].unique())
print('Intersection experiment and control groups: {}'.format(len(e&c)))

views: 6328 clicks: 1860
views who didn't click: 4468
clicks who didn't view: 0
Intersection experiment and control groups: 0


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [22]:
df['count'] = 1 #setting up a column to count lines
df.head()

,timestamp,id,group,action,count
0,2016-09-24 17:42:27.839496,804196,experiment,view,1
1,2016-09-24 19:19:03.542569,434745,experiment,view,1
2,2016-09-24 19:36:00.944135,507599,experiment,view,1
3,2016-09-24 19:59:02.646620,671993,control,view,1
4,2016-09-24 20:26:14.466886,536734,experiment,view,1


In [57]:
##### flatiron stats not included, so let's paste some of it here
####flatiron_stats.py
#flatiron_stats
import numpy as np
import scipy.stats as stats

def welch_t(a, b):
    
    """ Calculate Welch's t statistic for two samples. """

    numerator = a.mean() - b.mean()
    
    # “ddof = Delta Degrees of Freedom”: the divisor used in the calculation is N - ddof, 
    #  where N represents the number of elements. By default ddof is zero.
    
    denominator = np.sqrt(a.var(ddof=1)/a.size + b.var(ddof=1)/b.size)
    
    return np.abs(numerator/denominator)

def welch_df(a, b):
    
    """ Calculate the effective degrees of freedom for two samples. This function returns the degrees of freedom """
    
    s1 = a.var(ddof=1) 
    s2 = b.var(ddof=1)
    n1 = a.size
    n2 = b.size
    
    numerator = (s1/n1 + s2/n2)**2
    denominator = (s1/ n1)**2/(n1 - 1) + (s2/ n2)**2/(n2 - 1)
    
    return numerator/denominator


def p_value_welch_ttest(a, b, two_sided=False):
    """Calculates the p-value for Welch's t-test given two samples.
    By default, the returned p-value is for a one-sided t-test. 
    Set the two-sided parameter to True if you wish to perform a two-sided t-test instead.
    """
    t = welch_t(a, b)
    df = welch_df(a, b)
    
    p = 1-stats.t.cdf(np.abs(t), df)
    
    if two_sided:
        return 2*p
    else:
        return p

In [94]:
#Your code here

df.fillna(value=0)

con = df[df.group=='control'].groupby(['action'])
display('con',con.sum())
print("Control group: 932 clicks out of 3332 views; rate: {}".format(932/3332))
cc = 932
cv = 3332
#display(con.columns)

exp = df[df.group=='experiment'].groupby(['action'])
display('exp',exp.sum())
print("Experimental group: 928 clicks of 2996 views. rate: {}".format(928/2996))
ec = 928
ev = 2996

print('\n###### create new column for easy counting ####')
df['clickbool']=df.action.map(lambda x: 0 if x =='view' else 1)
display(df.clickbool.count(), df.clickbool.sum())
control=df[df.group == 'control']
control=control.groupby('id').max()
#display(a)
experiment=df[df.group =='experiment']
experiment=experiment.groupby('id').max()
display('pvalue:',p_value_welch_ttest(control.clickbool, experiment.clickbool))


'con'

,id,count,clickbool
action,,,
click,525683507,932,932
view,1874171236,3332,0


Control group: 932 clicks out of 3332 views; rate: 0.2797118847539016


'exp'

,id,count,clickbool
action,,,
click,528944090,928,928
view,1694962719,2996,0


Experimental group: 928 clicks of 2996 views. rate: 0.3097463284379172

###### create new column for easy counting ####


8188

1860

'pvalue:'

0.004466402814337078

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [96]:
c_rate = cc/cv
e_rate = ec/ev
expected_e_c = c_rate * ev #int(c_rate * ev)
expected_e_c

838.0168067226891

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [102]:
#Your code here
n = len(experiment)
#print(n)
p = c_rate
#print(p)
var = n * p * (1-p)
std = np.sqrt(var)
print(std)

24.568547907005815


In [104]:
z_score = (ec - expected_e_c)/std
print(z_score)

3.6625360854823588


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [106]:
#Your code here

import scipy.stats as stats
p_val = stats.norm.sf(z_score) #or 1 - stats.norm.cdf(z_score)
print(p_val)

0.00012486528006951198


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**
This approach showed an even more statistically significant result; both, however, were below an alpha of 0.01 so both were noteworthy.


## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.